**Portal Dados Abertos CVM**

https://dados.cvm.gov.br/dataset/

# **Carregamento das Bibliotecas**

In [ ]:
!pip install wget

In [2]:
from datetime import date, timedelta
import datetime 
import pandas as pd
import wget

# CARREGANDO O DATASET PELA 'URL'

O dataset utilizado pode ser visualizado e baixado do Portal Dados Abertos CVM (Comissão de Valores Mobiliários) do conjunto de dados: **"Fundos de Investimento: Documentos: Informe Diário"** e **"Fundos de Investimento: Informação Cadastral"**

## Fundos de Investimento: Documentos: Informe Diário || últimos 12 mês

O INFORME DIÁRIO é um demonstrativo que contém as seguintes informações do fundo, relativas à data de competência:

*   Valor total da carteira do fundo;
*   Patrimônio líquido;
*   Valor da cota;
*   Captações realizadas no dia;
*   Resgates pagos no dia;
*   Número de cotistas

(CVM, 2022)

In [46]:
lista_df =[]

for i in range(0, 13):  
  data = date.today() + pd.DateOffset(months= -i) #subtraindo a data atual pelos meses subsequentes
  ano = data.year #get no ano
  meses = data.month #get no mes

  if len(str(meses)) < 2:
    try:  
      mes = '0'+ str(meses) #acrescimo de zero na frete do meses com apenas 1 digito.
      fundos = wget.download('https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{}.zip'.format(ano,mes)) #descompactando o arquivo
      df = pd.read_csv(fundos, sep=';',encoding='ISO-8859-1') #lendo o arquivo
      lista_df.append(df) #concatenando as planilhas
    except OSError as err:
      print("Mes:",mes, "OS error: {0}".format(err))

  else:
    try:
      fundos = wget.download('https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{}.zip'.format(ano,meses)) #descompactando o arquivo
      df = pd.read_csv(fundos, sep=';',encoding='ISO-8859-1') #lendo o arquivo
      lista_df.append(df) #concatenando as planilhas
    except OSError as err:
      print('Mes:', mes,"OS error: {0}".format(err))    

df = pd.concat(lista_df) # Atribui os dados coletados de cada lista em um dataframe
informes_diarios = df.copy() #fazendo uma códia do dataframe

Mes: 06 OS error: HTTP Error 404: Not Found
Mes: 02 OS error: HTTP Error 404: Not Found
Mes: 01 OS error: HTTP Error 404: Not Found


**Observamos que existem algums arquivos que não estão disponiveis no portal da CVM na hora do get.**

## Fundos de Investimento - Conjunto Completo

Dados cadastrais de fundos de investimento estruturados e não estruturados (ICVM 555), tais como: CNPJ, data de registro e situação do fundo.

(CVM, 2022)

**Fundos de Investimento: Informação Cadastral**

In [47]:
try:  
  dados = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', sep=';',encoding='ISO-8859-1') #get no arquivo
except OSError as err:
  print("OS error: {0}".format(err))

cadastro_cvm = dados.copy() #fazendo uma cópia do dataframe
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2022-09-01,1071965.12,29.865451,1076999.37,0.0,0.00,1
1,FI,00.017.024/0001-53,2022-09-02,1072517.20,29.878939,1077485.76,0.0,0.00,1
2,FI,00.017.024/0001-53,2022-09-05,1073068.19,29.892127,1077961.34,0.0,0.00,1
3,FI,00.017.024/0001-53,2022-09-06,1070919.96,29.905284,1075522.95,0.0,2912.86,1
4,FI,00.017.024/0001-53,2022-09-08,1070905.42,29.918269,1075989.95,0.0,0.00,1


#EXPLORANDO E PREPARARANDO DOS DADOS

In [48]:
print('Total de Cadastro CVM:',cadastro_cvm.shape)
print('Total de informes diários:', informes_diarios.shape)

Total de Cadastro CVM: (68670, 39)
Total de informes diários: (4560772, 9)


## Normalização dos tipos dos valores



---


Informes Diários

In [50]:
#Transformando o type da serie DT_COMPTC em datetime

informes_diarios['DT_COMPTC'] = pd.to_datetime(informes_diarios['DT_COMPTC'])

#validando o type convertido
#informes_diarios.dtypes



---


Cadastro CVM

In [51]:
# Transformando o type das series em datetime

cadastro_cvm['DT_REG'] = pd.to_datetime(cadastro_cvm['DT_REG'])
cadastro_cvm['DT_CONST'] = pd.to_datetime(cadastro_cvm['DT_CONST'])
cadastro_cvm['DT_CANCEL'] = pd.to_datetime(cadastro_cvm['DT_CANCEL'])
cadastro_cvm['DT_INI_SIT'] = pd.to_datetime(cadastro_cvm['DT_INI_SIT'])
cadastro_cvm['DT_INI_EXERC'] = pd.to_datetime(cadastro_cvm['DT_INI_EXERC'])
cadastro_cvm['DT_INI_ATIV'] = pd.to_datetime(cadastro_cvm['DT_INI_ATIV'])
cadastro_cvm['DT_FIM_EXERC'] = pd.to_datetime(cadastro_cvm['DT_FIM_EXERC'])
cadastro_cvm['DT_INI_CLASSE'] = pd.to_datetime(cadastro_cvm['DT_INI_CLASSE'])
cadastro_cvm['DT_PATRIM_LIQ'] = pd.to_datetime(cadastro_cvm['DT_PATRIM_LIQ'])

#validando as transformações
#cadastro_cvm.dtypes

## Modelando os dados

In [52]:
#imprimindo as colunas do dataframe

informes_diarios.columns

Index(['TP_FUNDO', 'CNPJ_FUNDO', 'DT_COMPTC', 'VL_TOTAL', 'VL_QUOTA',
       'VL_PATRIM_LIQ', 'CAPTC_DIA', 'RESG_DIA', 'NR_COTST'],
      dtype='object')

-------------------------- Ordenando as series pelo maior número de cotistas --------------------------

In [53]:
informes_diarios = informes_diarios.sort_values(by='NR_COTST', ascending=False)
informes_diarios.head(5)

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
5668,FAPI,02.185.027/0001-21,2022-03-04,4.643428e+08,7.171034,4.641066e+08,6.200000e+02,9.538804e+04,4730510
5811,FAPI,02.185.027/0001-21,2022-08-15,4.827530e+08,7.489845,4.821160e+08,7.380000e+03,1.400570e+03,4721616
24913,FI,05.102.500/0001-58,2021-10-07,3.446449e+10,4.704745,3.446209e+10,1.737346e+09,1.435984e+09,1189547
24914,FI,05.102.500/0001-58,2021-10-08,3.439876e+10,4.705532,3.439636e+10,1.430685e+09,1.502179e+09,1188716
26257,FI,05.102.500/0001-58,2021-09-03,3.429615e+10,4.689395,3.429374e+10,1.435678e+09,1.764028e+09,1186005


--------------------------  Filtrando CNPJ com maior número de cotidas --------------------------

In [54]:
#seleciono as series
Total_cotistas = informes_diarios[['TP_FUNDO', 'CNPJ_FUNDO', 'NR_COTST']]

#Agrupando e sumarizando
Total_cotistas = Total_cotistas.groupby(['TP_FUNDO', 'CNPJ_FUNDO']).sum()

#imprimendo os três maiores números de cotistas
Total_cotistas.head(3)

NR_COTST
TP_FUNDO CNPJ_FUNDO                  
FACFIF   06.537.068/0001-90      3315
         07.408.147/0001-64     18720
FAPI     02.010.153/0001-45     83236

In [55]:
#Aqui podemos visualizar a media de cotidas

informes_diarios.describe()

,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
count,4.560772e+06,4.560772e+06,4.560772e+06,4.560772e+06,4.560772e+06,4.560772e+06
mean,4.182894e+08,1.842377e+04,4.180532e+08,2.874176e+06,2.873539e+06,1.031213e+03
std,2.851560e+09,5.007783e+06,2.852179e+09,8.696744e+07,8.836095e+07,1.561065e+04
min,-5.030349e+08,-7.401066e+06,-1.933905e+08,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.602720e+07,1.110737e+00,1.604848e+07,0.000000e+00,0.000000e+00,1.000000e+00
50%,4.519536e+07,2.213703e+00,4.523049e+07,0.000000e+00,0.000000e+00,2.000000e+00
75%,1.648724e+08,2.251624e+01,1.648922e+08,0.000000e+00,0.000000e+00,1.800000e+01
max,3.707869e+11,1.042557e+10,3.707870e+11,1.050720e+11,4.597324e+10,4.730510e+06


**/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////**

**Criando um novo dataframe pivotando os dados com o número maior que a media de cotidas por investimentos.**

**//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////**

In [58]:
 mediaCotistas = 1031

 fundos = informes_diarios[informes_diarios['NR_COTST'] >= mediaCotistas].pivot_table(index='DT_COMPTC', columns='CNPJ_FUNDO', 
                                                                               values=['VL_TOTAL','VL_QUOTA','VL_PATRIM_LIQ','CAPTC_DIA','RESG_DIA'])
 
 fundos.head(5)

CAPTC_DIA                                        \
CNPJ_FUNDO 00.068.305/0001-35 00.071.477/0001-68 00.180.995/0001-10   
DT_COMPTC                                                             
2021-09-01                0.0       2.669376e+09           62425.89   
2021-09-02                0.0       2.132567e+09           14655.83   
2021-09-03                0.0       2.398758e+09           57500.00   
2021-09-06                0.0       2.297415e+09           10000.00   
2021-09-08                0.0       2.593459e+09           10000.00   

                                                                     \
CNPJ_FUNDO 00.222.725/0001-24 00.222.816/0001-60 00.280.302/0001-60   
DT_COMPTC                                                             
2021-09-01                0.0          518971.52          443891.44   
2021-09-02            25300.0          251513.98          167736.00   
2021-09-03                0.0         1267419.08          579840.00   
2021-09-06                0.0         1703233.58          497000.00   
2021-09-08            20000.0         2472182.96          703843.10   

                                                                     \
CNPJ_FUNDO 00.306.278/0001-91 00.322.699/0001-06 00.360.293/0001-18   
DT_COMPTC                                                             
2021-09-01          889908.57         2734216.97           78500.00   
2021-09-02         1727361.99         3932000.00           35443.00   
2021-09-03         1395123.79          954367.84          113503.04   
2021-09-06         2447926.13          406000.00           17810.00   
2021-09-08         2206729.57         4464196.33          149834.87   

                               ...           VL_TOTAL                     \
CNPJ_FUNDO 00.398.561/0001-90  ... 67.976.449/0001-60 68.599.141/0001-06   
DT_COMPTC                      ...                                         
2021-09-01            6200.00  ...       2.405464e+08       4.191982e+09   
2021-09-02           96100.00  ...       2.343136e+08       4.182134e+09   
2021-09-03           53725.96  ...       2.347302e+08       4.171582e+09   
2021-09-06            8516.76  ...       2.364637e+08       4.158354e+09   
2021-09-08          115335.95  ...       2.273387e+08       4.149375e+09   

                                                                     \
CNPJ_FUNDO 68.623.479/0001-56 68.670.512/0001-07 73.899.759/0001-21   
DT_COMPTC                                                             
2021-09-01       8.067602e+08       2.930561e+08       1.630223e+08   
2021-09-02       8.067641e+08       2.866619e+08       1.592753e+08   
2021-09-03       8.067932e+08       2.903687e+08       1.594538e+08   
2021-09-06       8.069540e+08       2.862157e+08       1.605868e+08   
2021-09-08       8.066045e+08       2.828803e+08       1.540130e+08   

                                                                     \
CNPJ_FUNDO 74.326.471/0001-20 88.002.696/0001-36 88.198.056/0001-43   
DT_COMPTC                                                             
2021-09-01         6095772.16        63886054.98       2.147216e+08   
2021-09-02         6100636.34        62489828.20       2.101191e+08   
2021-09-03         6089002.45        62502637.83       2.074360e+08   
2021-09-06         6089127.45        62991708.45       2.089292e+08   
2021-09-08         6052003.99        60581294.97       2.007772e+08   

                                                  
CNPJ_FUNDO 97.519.703/0001-62 97.519.794/0001-36  
DT_COMPTC                                         
2021-09-01       4.209641e+09       1.781011e+08  
2021-09-02       4.216599e+09       1.780375e+08  
2021-09-03       4.228047e+09       1.777748e+08  
2021-09-06       4.243262e+09       1.777129e+08  
2021-09-08       4.234551e+09       1.777037e+08  

[5 rows x 8860 columns]

In [59]:
fundos.shape

(198, 8860)

### SubConjuntos

In [ ]:
#fundos['VL_TOTAL']
#fundos['VL_QUOTA']
#fundos['VL_PATRIM_LIQ']
#fundos['CAPTC_DIA']
#fundos['RESG_DIA']

# Normalizando os dados de cotas para efeitos comparativos de fundos de investimentos

Para sabermos a variação do dia referente ao primeiro dia do mês, consideramos a primeira cota do mês como divisor e as demais cotas subsequentes como dividendo

* Os valores estão em porcentagem

Para a base de cálculo da variação, consideramos valor da cota atribuido na data mais antiga e comparamos com os demais valores até os dias atuais.

In [60]:
#dividimos o valor da primeira posição (pois o dataframe esta em ordem decrescente) pelo valor das demais linha da coluna
cotas = ((fundos['VL_QUOTA'] / fundos['VL_QUOTA'].iloc[0])-1)*100
cotas.head(5)

CNPJ_FUNDO,00.068.305/0001-35,00.071.477/0001-68,00.180.995/0001-10,00.222.725/0001-24,00.222.816/0001-60,00.280.302/0001-60,00.306.278/0001-91,00.322.699/0001-06,00.360.293/0001-18,00.398.561/0001-90,...,67.976.449/0001-60,68.599.141/0001-06,68.623.479/0001-56,68.670.512/0001-07,73.899.759/0001-21,74.326.471/0001-20,88.002.696/0001-36,88.198.056/0001-43,97.519.703/0001-62,97.519.794/0001-36
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,
2021-09-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2021-09-02,0.019695,0.012410,-0.008190,-0.001317,-0.008549,0.005348,0.021140,0.023414,0.358631,-1.354989,...,-2.278906,0.014289,0.019738,-1.613873,-2.264844,0.383798,-2.184117,-2.125842,0.030286,0.024054
2021-09-03,0.032734,0.024732,0.016775,0.018955,0.015826,0.023101,0.039908,0.043615,0.286971,-1.924077,...,-2.077008,0.031122,0.032834,-0.204951,-2.088193,0.101701,-2.161930,-2.374644,0.057364,0.048505
2021-09-06,0.049600,0.037218,0.045773,0.049836,0.046865,0.046794,0.060614,0.065950,0.162977,-1.676405,...,-1.299097,0.045403,0.049748,-1.613040,-1.340404,0.176855,-1.396138,-1.391409,0.084887,0.071472
2021-09-08,0.053286,0.049811,0.041988,0.047633,0.043416,0.037740,0.080813,0.089191,2.289822,-5.583695,...,-5.037804,0.059120,0.053497,-3.918129,-5.046299,1.118338,-5.169260,-4.877233,0.112191,0.090334


## Ranking dos 10 Fundos de Investimento com os melhores rendimentos nos últimos 12 meses

In [61]:
ranking = pd.DataFrame() #crio um dataframe vazio
ranking['retorno(%)'] = cotas.iloc[-1].sort_values(ascending=False)[:10] #chamamos a linha pelo último indice, ordenamos em decrescente e selecionamos os 10
ranking

,retorno(%)
CNPJ_FUNDO,
10.590.125/0001-72,100.084664
03.917.778/0001-58,98.951678
10.869.628/0001-81,98.710987
03.555.959/0001-81,97.958398
38.267.929/0001-54,97.794266
03.915.903/0001-90,97.548521
02.998.275/0001-91,97.518613
03.917.776/0001-69,97.476414
03.922.006/0001-04,97.306222


Buscando dados dos Fundos de Investimento pelo CNPJ, através da fonte de dados do **'Cadastro da CVM'**

In [62]:
for cnpj in ranking.index:
  fundo = cadastro_cvm[cadastro_cvm['CNPJ_FUNDO'] == cnpj]
  ranking.at[cnpj, 'Carteira de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  ranking.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  ranking.at[cnpj, 'VL_PATRIM_LIQ'] = fundo['VL_PATRIM_LIQ'].values[0]

ranking

,retorno(%),Carteira de Investimento,Classe,VL_PATRIM_LIQ
CNPJ_FUNDO,,,,
10.590.125/0001-72,100.084664,BRADESCO FUNDO DE INVESTIMENTO EM AÇÕES CIELO,Fundo de Ações,2.056724e+07
03.917.778/0001-58,98.951678,ITAÚ AÇÕES PETROBRÁS - FUNDO DE INVESTIMENTO,Fundo de Ações,3.354232e+08
10.869.628/0001-81,98.710987,BB AÇÕES CIELO FUNDO DE INVESTIMENTO,Fundo de Ações,6.657649e+07
03.555.959/0001-81,97.958398,CAIXA FMP - FGTS PETROBRAS IV,FMP-FGTS,6.181693e+08
38.267.929/0001-54,97.794266,ITAÚ INDEX PETROBRÁS AÇÕES FUNDO DE INVESTIMEN...,Fundo de Ações,1.766529e+08
03.915.903/0001-90,97.548521,CAIXA FMP FGTS PETROBRAS III - CAIXA FMP FGTS ...,FMP-FGTS,7.046784e+08
02.998.275/0001-91,97.518613,BRADESCO FMP - FGTS PETROBRAS,FMP-FGTS,4.135031e+08
03.917.776/0001-69,97.476414,ITAU PETROBRAS FMP - FGTS,FMP-FGTS,2.679880e+08
03.922.006/0001-04,97.306222,BRADESCO H FUNDO DE INVESTIMENTO AÇÕES PETROBRAS,Fundo de Ações,4.919031e+07


## Os 10 Fundos de Investimento com os menores rendimento nos últimos 12 meses

In [63]:
menorRendimento = pd.DataFrame()
menorRendimento['retorno(%)'] = cotas.iloc[-1].sort_values(ascending=True)[:10] #chamamos a linha pelo último indice, ordenamos em crescente e selecionamos os 10
menorRendimento

,retorno(%)
CNPJ_FUNDO,
41.033.759/0001-58,-77.107256
24.874.367/0001-00,-65.859415
34.218.748/0001-87,-65.457315
42.295.133/0001-82,-64.470603
41.545.885/0001-91,-63.749683
34.218.740/0001-10,-62.335484
34.218.661/0001-00,-62.265433
34.218.752/0001-45,-57.150310
33.736.845/0001-07,-56.548605


Buscando dados dos Fundos de Investimento pelo CNPJ, através da fonte de dados do **'Cadastro da CVM'**

In [64]:
for cnpj in menorRendimento.index:
  fundo = cadastro_cvm[cadastro_cvm['CNPJ_FUNDO'] == cnpj]
  menorRendimento.at[cnpj, 'Carteira de Investimento'] = fundo['DENOM_SOCIAL'].values[0]
  menorRendimento.at[cnpj, 'Classe'] = fundo['CLASSE'].values[0]
  menorRendimento.at[cnpj, 'VL_PATRIM_LIQ'] = fundo['VL_PATRIM_LIQ'].values[0]


menorRendimento

,retorno(%),Carteira de Investimento,Classe,VL_PATRIM_LIQ
CNPJ_FUNDO,,,,
41.033.759/0001-58,-77.107256,VITREO CRIPTO DEFI FI EM COTAS DE FUNDOS DE IN...,Fundo Multimercado,1.339245e+07
24.874.367/0001-00,-65.859415,INTER + IBOVESPA ATIVO FUNDO DE INVESTIMENTO E...,Fundo de Ações,5.174836e+07
34.218.748/0001-87,-65.457315,VITREO CANNABIS ATIVO FUNDO DE INVESTIMENTO MU...,Fundo Multimercado,1.713381e+07
42.295.133/0001-82,-64.470603,INGENIOUS VITREO CANNABIS ATIVO FUNDO DE INVES...,Fundo Multimercado,1.820567e+06
41.545.885/0001-91,-63.749683,VITREO COIN DEFI FUNDO DE INVESTIMENTO MULTIME...,Fundo Multimercado,1.572298e+07
34.218.740/0001-10,-62.335484,TREND CANNABIS FUNDO DE INVESTIMENTO MULTIMERCADO,Fundo Multimercado,3.600824e+07
34.218.661/0001-00,-62.265433,VITREO CANABIDIOL FUNDO DE INVESTIMENTO EM AÇÕ...,Fundo de Ações,5.053408e+07
34.218.752/0001-45,-57.150310,VITREO EXPONENCIAL FUNDO DE INVESTIMENTO EM AÇ...,Fundo de Ações,2.456327e+07
33.736.845/0001-07,-56.548605,HASHDEX 100 NASDAQ CRYPTO INDEX FUNDO DE INVES...,Fundo Multimercado,9.789890e+07


----------- Fim -----------